# Побудова оцінок та довірчих інтервалів

Будуємо довірчі інтервали для `n = 100, 10_000, 1_000_000`
Рівень довіри = `0,99` (рівень значущості = `0.01`

#### Довірчі інтервали для м.с. та дисперсії у випадку нормально розподіленої генеральної сукупності.
##### Перша частина леми 1. (Пункт А):
Вибіркове математичне сподівання: $\hat{a}_{n}=\frac {1}{n}\sum^{n}_{k=1}X_k$
Вибіркова дисперсія: $\hat{\sigma}^2_{n}=\frac {1}{n}\sum^{n}_{k=1}(X_k-\hat{a}_{n})^2$
Дов інтервал $a \in a_n \pm \frac{z_{\gamma}  \sigma_n}{\sqrt{n}}$
К-ть ітерацій: $n$

In [2]:
import math

import numpy as np
from scipy.stats import chi2, t, norm

In [3]:
def calc_sample_variance(samples, sample_mean: float):
    squared_variance = 0.
    for sample in samples:
        squared_variance += (sample - sample_mean) ** 2

    squared_variance /= (len(samples) - 1)
    return math.sqrt(squared_variance)


def calc_sample_mean(samples):
    mean = 0.
    for sample in samples:
        mean += sample
    return mean / len(samples)


def calc_s_squared(samples, sample_mean):
    s_squared = 0
    for sample in samples:
        s_squared += (sample - sample_mean) ** 2

    s_squared /= (len(samples) - 1)
    return s_squared


# generating samples with help of N(0,1) distribution
def gen_samples(size: int):
    samples = size * [None]
    for i in range(size):
        samples[i] = np.random.normal(0, 1)
    return samples

In [4]:
n_arr = [100, 10_000, 1_000_000]  # for calculating Confidence Intervals
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m

samples_list = len(n_arr) * [None]
alfa = 0.01
gama = 1 - alfa
z_gama = 2.575  # norm distro 0.99 quantile

Q_m = lambda q, n: np.sum(q) / n
# calculating variance in integral methods
# незміщена?
calc_variance = lambda q, n, Q: (1 / (n-1)) * (np.sum([qi**2 for qi in q]) - n * Q ** 2)

eta = lambda x: 1 / x - 1
xi_i = lambda x: -1 * np.log(x)

# calculating 0.99 quantiles
print(f"samples_num_ci:{n_arr}\nm_arr:{m_arr}")

samples_num_ci:[100, 10000, 1000000]
m_arr:[1, 10, 100, 1000, 10000]


In [5]:
# calculating Confidence Interval for the EXPECTATION(sigma^2) under the assumption:
# - observed random value ~ N(0,1)
# - sample variance is unknown
def calculate_ci_1(samples):
    sample_mean = calc_sample_mean(samples)
    n_sq_root = math.sqrt(len(samples))
    s_squared = calc_s_squared(samples, sample_mean)

    g = t.ppf((1 + gama) / 2, df=len(samples) - 1)
    return [sample_mean - math.sqrt(s_squared) * g / n_sq_root, sample_mean + math.sqrt(s_squared) * g / n_sq_root]


# calculating Confidence Interval for the EXPECTATION under assumption:
# - observed value distribution is unknown
# asymptotic confidence interval
def calculate_ci_2(samples, gama):
    sample_mean = calc_sample_mean(samples)
    sample_variance = calc_sample_variance(samples, sample_mean)

    n_sq_root = math.sqrt(len(samples))

    c = norm.ppf((1 + gama) / 2)
    return [sample_mean - sample_variance * c / n_sq_root, sample_mean + sample_variance * c / n_sq_root]


# calculating Confidence Interval for the variance under assumption:
# - observed random value ~ N(0,1)
def calculate_ci_3(samples, gama):
    sample_mean = calc_sample_mean(samples)
    g_1 = chi2.ppf((1 - gama) / 2, df=len(samples) - 1)
    g_2 = chi2.ppf((1 + gama) / 2, df=len(samples) - 1)

    squared_difference = 0
    for sample in samples:
        squared_difference = (sample - sample_mean) ** 2

    return [squared_difference / g_2, squared_difference / g_1]

In [6]:

def calculate_2_integral():
    pass

In [7]:
def calculate_3_integral():
    pass

In [8]:
for i, sample_size in enumerate(n_arr):
    samples_list[i] = gen_samples(sample_size)
    sample_mean = calc_sample_mean(samples_list[i])
    sample_variance = calc_sample_variance(samples_list[i], sample_mean)

    ci_1 = calculate_ci_1(samples_list[i])
    print('\n[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_1[0]} ; {ci_1[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_1[1] - ci_1[0]}')

    ci_2 = calculate_ci_2(samples_list[i], gama)
    print('\n[2] Довірчий інтервал для мат. спод. M при невідомому розподілі')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_2[0]} ; {ci_2[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_2[1] - ci_2[0]}')

    ci_3 = calculate_ci_3(samples_list[i], gama)
    print('\n[3] Lовірчий інтервал для дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркова дисперсія = {sample_variance}')
    print(f'Довірчий інтервал:[{ci_3[0]} ; {ci_3[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_3[1] - ci_3[0]}')


[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 100
Вибіркове математичне сподівання: 0.007650077666394552
Довірчий інтервал:[-0.23832405245161203 ; 0.25362420778440115]
Довжина довірчого інтервалу: 0.4919482602360132

[2] Довірчий інтервал для мат. спод. M при невідомому розподілі
Кількість спостережень: n = 100
Вибіркове математичне сподівання: 0.007650077666394552
Довірчий інтервал:[-0.2335873789237772 ; 0.24888753425656632]
Довжина довірчого інтервалу: 0.4824749131803435

[3] Lовірчий інтервал для дисперсії
Кількість спостережень: n = 100
Вибіркова дисперсія = 0.9365428689618603
Довірчий інтервал:[0.0005792725500497568 ; 0.0012105112164171533]
Довжина довірчого інтервалу: 0.0006312386663673966

[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 10000
Вибіркове математичне сподівання: -0.002100346496743533
Довірчий інтервал:[-0.028097948932395964 ; 0.023897255938908894]
Довжина довірчого інтервалу: 0

# Метод 1
Будемо обраховувати інтеграл за допомогою методу Монте-Карло:
$$Q_m = \bf{M}(I(\eta > \xi_{1} + \ldots + \xi_m))$$

(варто підмітити, що із збільшенням m значення інтеграла зменшується у $\sqrt{m}$ разів)

In [9]:
q_1 = lambda eta, samples: 1 if eta > np.sum(samples) else 0
m_arr = [1, 10, 100, 1_000]  # for calculating Q_m
n_0 = 5000  #

for m in m_arr:
    indicators = []

    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_1(eta(np.random.random(1)), samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators),  Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6892
Вибіркова дисперсія: 0.21424620924184834
Асимптотичний довірчий інтервал: [0.6723387933677308, 0.7060612066322692]
Кількість спостережень: 5000
N*: 29907.2909474524

Кількість зразків: 10
Оцінка Q: 0.1702
Вибіркова дисперсія: 0.1412602120424085
Асимптотичний довірчий інтервал: [0.1565087845627959, 0.18389121543720408]
Кількість спостережень: 5000
N*: 323336.8545036857

Кількість зразків: 100
Оцінка Q: 0.018
Вибіркова дисперсія: 0.017679535907181434
Асимптотичний довірчий інтервал: [0.013156408508179084, 0.022843591491820914]
Кількість спостережень: 5000
N*: 3618097.925140584

Кількість зразків: 1000
Оцінка Q: 0.0022
Вибіркова дисперсія: 0.002195599119823965
Асимптотичний довірчий інтервал: [0.0004930979443842037, 0.003906902055615797]
Кількість спостережень: 5000
N*: 30078914.078270197



Метод 2

In [10]:

m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 100
q_2 = lambda samples: 1/(1+np.sum(samples))


for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))
    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)
    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.7052984262770251
Вибіркова дисперсія: 0.020957467068561244
Асимптотичний довірчий інтервал: [0.6680089335184459, 0.7425879190356038]
Кількість спостережень: 100
N*: 2793.4919843856023

Кількість зразків: 10
Оцінка Q: 0.16890659571484348
Вибіркова дисперсія: 0.0006863813651453715
Асимптотичний довірчий інтервал: [0.16215821117733586, 0.1756549802523512]
Кількість спостережень: 100
N*: 1595.2425796634204

Кількість зразків: 100
Оцінка Q: 0.019869172763370614
Вибіркова дисперсія: 1.2839048616219395e-06
Асимптотичний довірчий інтервал: [0.019577306766680198, 0.02016103876006105]
Кількість спостережень: 100
N*: 215.63921298599803

Кількість зразків: 1000
Оцінка Q: 0.001997917386875396
Вибіркова дисперсія: 1.5811448361326306e-09
Асимптотичний довірчий інтервал: [0.00198767495918515, 0.00200815981456564]
Кількість спостережень: 100
N*: 26.26461675403712

Кількість зразків: 10000
Оцінка Q: 0.000199686227044988
Вибіркова дисперсія: 1.0601029802885885e-12
Асимпто

Метод 4

In [11]:
m_arr = [1, 10, 100, 1_000, 10_000] # for calculation Q_m
n_0 = 100
q_4 = lambda m, x, samples : (1 / (m - 1)) * ((np.sum(samples)) / (1 + np.sum(samples)))

for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))
        
    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)
    
    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.7063990217889996
Вибіркова дисперсія: 0.019033112699155988
Асимптотичний довірчий інтервал: [0.6708627396600985, 0.7419353039179005]
Кількість спостережень: 100
N*: 2529.088970387316

Кількість зразків: 10
Оцінка Q: 0.16919776724242563
Вибіркова дисперсія: 0.0004736947955368792
Асимптотичний довірчий інтервал: [0.16359159565315975, 0.17480393883169146]
Кількість спостережень: 100
N*: 1097.1444852165416

Кількість зразків: 100
Оцінка Q: 0.019637147042019187
Вибіркова дисперсія: 1.1777245986145589e-06
Асимптотичний довірчий інтервал: [0.019357610265614257, 0.01991668381842413]
Кількість спостережень: 100
N*: 202.50764751781026

Кількість зразків: 1000
Оцінка Q: 0.001999337084446198
Вибіркова дисперсія: 1.264265449150741e-09
Асимптотичний довірчий інтервал: [0.001990178334241417, 0.00200849583465098]
Кількість спостережень: 100
N*: 20.971074982251917

Кількість зразків: 10000
Оцінка Q: 0.00019987703798679605
Вибіркова дисперсія: 1.7390227275082123e-12
Асим